In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from time import time

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
happy = pd.read_csv('data/happy.csv', encoding='utf-8')
text = happy['text'][pd.notnull(happy['text'])]
# Add extra space at the end of each post
# so that after concatenation they remain separated
chars = text.map(lambda s: list(s) + [u' '])
# Flatten chars
chars = [item for sublist in chars for item in sublist]
# Replace space by the special character underscore
chars = [u'_' if x == ' ' else x for x in chars]

# Map char to id
df_chars = pd.DataFrame({'char': chars})
char_count = df_chars.groupby('char')['char'].count()
reduced_chars = char_count[char_count > 10000]
char_to_id = dict(zip(reduced_chars.index, np.arange(reduced_chars.shape[0], dtype=int)))
unknown_char_id = len(char_to_id)

train_data = np.array(map(lambda c: char_to_id.get(c, unknown_char_id), chars))[:5000000]
char_to_id[u'<unk>'] = len(char_to_id)

In [3]:
np.savetxt('data_chars/happy_chars.txt', train_data, fmt='%d')
np.save('data_chars/char_to_id.npy', char_to_id)

In [4]:
# Only contains 5M chars
full_data = np.loadtxt('data_chars/happy_chars.txt', dtype=int)
char_to_id = np.load('data_chars/char_to_id.npy').item()
id_to_char = {i: k for k, i in char_to_id.iteritems()}

# Split into train/val/test 80/10/10
train_split = (int)(full_data.shape[0] * 0.8)
val_split = (int)(full_data.shape[0] * 0.9)
train_data = full_data[:train_split]
valid_data = full_data[train_split:val_split]
test_data = full_data[val_split:]

In [3]:
char_to_id = np.load('data_chars/char_to_id.npy').item()

In [6]:
char_to_id

{u'!': 0,
 u'#': 1,
 u'(': 2,
 u')': 3,
 u',': 4,
 u'-': 5,
 u'.': 6,
 u'/': 7,
 u'0': 8,
 u'1': 9,
 u'2': 10,
 u'3': 11,
 u'4': 12,
 u'5': 13,
 u'6': 14,
 u'7': 15,
 u':': 16,
 u'<unk>': 95,
 u'?': 17,
 u'@': 18,
 u'A': 19,
 u'B': 20,
 u'C': 21,
 u'D': 22,
 u'E': 23,
 u'F': 24,
 u'G': 25,
 u'H': 26,
 u'I': 27,
 u'J': 28,
 u'K': 29,
 u'L': 30,
 u'M': 31,
 u'N': 32,
 u'O': 33,
 u'P': 34,
 u'R': 35,
 u'S': 36,
 u'T': 37,
 u'U': 38,
 u'V': 39,
 u'W': 40,
 u'Y': 41,
 u'_': 42,
 u'a': 43,
 u'b': 44,
 u'c': 45,
 u'd': 46,
 u'e': 47,
 u'f': 48,
 u'g': 49,
 u'h': 50,
 u'i': 51,
 u'j': 52,
 u'k': 53,
 u'l': 54,
 u'm': 55,
 u'n': 56,
 u'o': 57,
 u'p': 58,
 u'q': 59,
 u'r': 60,
 u's': 61,
 u't': 62,
 u'u': 63,
 u'v': 64,
 u'w': 65,
 u'x': 66,
 u'y': 67,
 u'z': 68,
 u'\u0430': 69,
 u'\u0432': 70,
 u'\u0434': 71,
 u'\u0435': 72,
 u'\u0438': 73,
 u'\u043a': 74,
 u'\u043b': 75,
 u'\u043c': 76,
 u'\u043d': 77,
 u'\u043e': 78,
 u'\u043f': 79,
 u'\u0440': 80,
 u'\u0441': 81,
 u'\u0442': 82,
 u'\u0443': 

In [8]:
from rnn_chars import main
main()

Training:
Epoch: 1
0% perplexity = 94.860, accuracy = 0.023, speed = 1244 cps
10% perplexity = 38.927, accuracy = 0.121, speed = 3314 cps
20% perplexity = 34.707, accuracy = 0.125, speed = 3327 cps
30% perplexity = 33.657, accuracy = 0.124, speed = 3324 cps
40% perplexity = 32.671, accuracy = 0.124, speed = 3191 cps
50% perplexity = 32.244, accuracy = 0.125, speed = 3165 cps
60% perplexity = 32.019, accuracy = 0.124, speed = 3140 cps
70% perplexity = 31.944, accuracy = 0.124, speed = 3088 cps
80% perplexity = 31.817, accuracy = 0.124, speed = 3067 cps
90% perplexity = 31.682, accuracy = 0.124, speed = 3063 cps
100% perplexity = 31.522, accuracy = 0.124, speed = 3055 cps
Generated characters:
h e odoea.ey#rydоo<unk>aotahhphhwisddbrtewS<unk>lm#t’lia<unk>wOiue<unk>nseeh#hnhnadeyp.rcneohluo<unk>ndbrdPi#wtocreebgjeaececnabnhmmG2tIl<unk>tcie#opt�s yyr(m eti#ympab!oessl eynytoysrоAwiwewahhaaev rlrimopa!beeecagtredhunse.enyc@oeoageeoideaum#gseferwtmioys#pmayhrylo<unk>mpo.icofaobiearioa.mvonfoi

KeyboardInterrupt: 